In [ ]:
%pylab inline

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
from numpy.fft import fft, ifft

def real_space_vectors(N, L):
    return linspace(0, L, N+1)[:-1]

def reciprocal_space_vectors(N, L):
    G = fftfreq(N, d=1./N) * 2*pi/L
    if N%2 == 0: G[N/2] *= -1
    G2 = G**2
    return G, G2

def real2fourier(X):
    return fft(X)/size(X)

def fourier2real(X):
    return ifft(X)*size(X)

In [ ]:
def f(N):
    L = 2*pi
    X = real_space_vectors(N, L)
    G, G2 = reciprocal_space_vectors(N, L)

    x = 0
    M = 20
    for i in range(1, M, 2):
        x += cos(i*X) * (M-i)

    y = real2fourier(x)
    figure(figsize=(12, 4))
    subplot(121)
    xlim([0, L])
    plot(X, x)
    subplot(122)
    plot(G, abs(y), "o")
    xlim([-25, 25])
    ylim([0, 10])

In [ ]:
interact(f, N=widgets.IntSlider(min=1,max=99,step=1,value=45));

In [ ]:
N = 20
L = 10.
X = real_space_vectors(N, L)
G, G2 = reciprocal_space_vectors(N, L)

x = exp(-10*(X-L/2)**2)

y = real2fourier(x)

y[sqrt(G2) > N/3 *2*pi/L] = 0
xx = fourier2real(y).real

figure(figsize=(12, 4))
subplot(121)
xlim([0, L])
plot(X, x)
plot(X, xx.real, "o")
subplot(122)
plot(G, abs(y), "o")
xlim([-20, 20])
ylim([0, 0.06])

#N = 100
#L = 10.
#X = real_space_vectors(N, L)
#G, G2 = reciprocal_space_vectors(N, L)

#x = exp(-10*(X-L/2)**2)
x = xx**2

y = real2fourier(x)
subplot(121)
plot(X, x)
subplot(122)
plot(G, abs(y), "o")

V = y[:]
V[1:] /= G2[1:]
V[0] = 0
plot(G, abs(y), "o")
E = sum(V*y).real
E_ref = 2.15122296713963594e-02
print "%.2e" % abs(E-E_ref)

In [ ]:
D = []
E_ref = 2.15122296713963594e-02
E_ref = 0.01440729580367173
for N in range(4, 300, 2):
    L = 10.
    X = real_space_vectors(N, L)
    G, G2 = reciprocal_space_vectors(N, L)

    psi = exp(-10*(X-L/2)**2)
    psiG = real2fourier(psi)

    psiG_cut = psiG[:]
    psiG_cut[sqrt(G2) > N/4. *2*pi/L] = 0
    psi_cut = fourier2real(psiG_cut)
    
    n = psi**3
    n_cut = psi_cut**3
    
    nG = real2fourier(n)
    nG_cut = real2fourier(n_cut)
    
    VG = nG[:]
    VG[0] = 0; VG[1:] /= G2[1:]
    E = sum(VG*nG).real
    VG_cut = nG_cut[:]
    VG_cut[0] = 0; VG_cut[1:] /= G2[1:]
    E_cut = sum(VG_cut*nG_cut).real
    #print N, "%.2e" % abs(E-E_ref), "%.2e" % abs(E_cut-E_ref)
    D.append([N, E, E_cut])
    
D = array(D)
semilogy(D[:, 0], abs(D[:,1]-E_ref), label="no cut")
semilogy(D[:, 0], abs(D[:,2]-E_ref), label="cut")
semilogy(D[:, 0]/2, abs(D[:,2]-E_ref), label="cut, plot N/2")
legend()
grid()
xlim([0, 200])
show()